In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielense20m/rating.csv
/kaggle/input/movielense20m/movie.csv


In [64]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv("/kaggle/input/movielense20m/movie.csv")
    rating = pd.read_csv("/kaggle/input/movielense20m/rating.csv")
    df = movie.merge(rating, how="left", on="movieId")
    comment_count = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_count[comment_count["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

In [65]:
user_movie_df = create_user_movie_df()

In [66]:
user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)

In [68]:
random_user

28941

### Öneri yapılacak kullanıcının izlediği filmlerin belirlenmesi

In [69]:
random_user_df = user_movie_df[user_movie_df.index == random_user]

In [70]:
random_user_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
28941.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

In [72]:
movies_watched

['Ace Ventura: Pet Detective (1994)',
 'Ace Ventura: When Nature Calls (1995)',
 'Aladdin (1992)',
 'American President, The (1995)',
 'Apollo 13 (1995)',
 'Babe (1995)',
 'Bullets Over Broadway (1994)',
 'Clueless (1995)',
 'Disclosure (1994)',
 'Forrest Gump (1994)',
 'Four Weddings and a Funeral (1994)',
 'Home Alone (1990)',
 'Jurassic Park (1993)',
 'Like Water for Chocolate (Como agua para chocolate) (1992)',
 'Little Women (1994)',
 "Mr. Holland's Opus (1995)",
 'Mrs. Doubtfire (1993)',
 'Much Ado About Nothing (1993)',
 "Muriel's Wedding (1994)",
 'Nine Months (1995)',
 'Operation Dumbo Drop (1995)',
 'Piano, The (1993)',
 'Postman, The (Postino, Il) (1994)',
 'Ready to Wear (Pret-A-Porter) (1994)',
 'Remains of the Day, The (1993)',
 'Sabrina (1995)',
 "Schindler's List (1993)",
 'Secret Garden, The (1993)',
 'Sense and Sensibility (1995)',
 'Shadowlands (1993)',
 'Silence of the Lambs, The (1991)',
 'Star Trek: Generations (1994)',
 'Stargate (1994)']

In [73]:
user_movie_df.loc[user_movie_df.index == random_user, user_movie_df.columns=="Mr. Holland's Opus (1995)"]

title,Mr. Holland's Opus (1995)
userId,
28941.0,3.0


In [74]:
user_movie_df.loc[user_movie_df.index == random_user, user_movie_df.columns=='Stargate (1994)']

title,Stargate (1994)
userId,
28941.0,1.0


In [75]:
len(movies_watched)

33

### Aynı filmleri izleyen diğer kullanıcılar

In [76]:
movies_watched_df = user_movie_df[movies_watched]

In [77]:
movies_watched_df.head()

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN
5.0,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,5.0,3.0,NaN,3.0,NaN,4.0


In [78]:
user_movie_count = movies_watched_df.T.notnull().sum()

In [79]:
user_movie_count

userId
1.0          1
2.0          2
3.0          4
4.0          6
5.0         11
            ..
138489.0     1
138490.0     7
138491.0     0
138492.0     2
138493.0     9
Length: 138493, dtype: int64

In [80]:
user_movie_count = user_movie_count.reset_index()

In [81]:
user_movie_count

,userId,0
0,1.0,1
1,2.0,2
2,3.0,4
3,4.0,6
4,5.0,11
...,...,...
138488,138489.0,1
138489,138490.0,7
138490,138491.0,0
138491,138492.0,2


In [82]:
user_movie_count.columns = ["userId", "movie_count"]

In [83]:
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False)

,userId,movie_count
94230,94231.0,33
100398,100399.0,33
118204,118205.0,33
15918,15919.0,33
124051,124052.0,33
...,...,...
79214,79215.0,21
79174,79175.0,21
9105,9106.0,21
78515,78516.0,21


In [84]:
user_movie_count[user_movie_count["movie_count"] == 33].count()

userId         17
movie_count    17
dtype: int64

In [85]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]

In [86]:
users_same_movies

129          130.0
155          156.0
157          158.0
183          184.0
294          295.0
            ...   
138207    138208.0
138278    138279.0
138381    138382.0
138414    138415.0
138482    138483.0
Name: userId, Length: 3202, dtype: float64

In [ ]:
# perc = len(movies_watched) * 0
# users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]

### Benzerliklerin Belirlenmesi

In [42]:
# Öneri yapılacak kullanıcı ile en benzer davranışlı kullanıcının belirlenmesi

In [87]:
random_user_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
28941.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
         random_user_df[movies_watched]])

In [89]:
final_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
130.0,4.0,3.0,NaN,3.0,3.0,NaN,NaN,3.0,5.0,5.0,...,NaN,3.0,NaN,5.0,NaN,NaN,3.0,5.0,NaN,3.0
156.0,3.0,NaN,NaN,5.0,5.0,3.0,NaN,NaN,4.0,5.0,...,NaN,NaN,4.0,5.0,NaN,4.0,4.0,5.0,3.0,4.0
158.0,2.0,1.0,4.0,4.0,3.0,5.0,NaN,4.0,NaN,5.0,...,NaN,5.0,3.0,5.0,5.0,4.0,5.0,5.0,NaN,NaN
184.0,2.0,3.0,3.0,4.0,4.0,NaN,3.0,NaN,4.0,3.0,...,NaN,4.0,4.0,5.0,4.0,NaN,4.0,5.0,3.0,4.0
295.0,NaN,NaN,3.0,3.0,3.0,3.0,3.0,2.0,NaN,4.0,...,NaN,3.0,3.0,4.0,3.0,4.0,NaN,4.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138279.0,3.0,NaN,NaN,3.0,5.0,5.0,5.0,4.0,NaN,5.0,...,NaN,5.0,NaN,5.0,NaN,4.0,NaN,5.0,3.0,3.0
138382.0,1.0,1.0,4.0,2.0,3.0,5.0,NaN,4.0,3.0,5.0,...,NaN,NaN,3.0,NaN,3.0,3.0,NaN,5.0,NaN,4.0
138415.0,1.0,NaN,3.0,4.0,3.0,5.0,NaN,NaN,NaN,4.0,...,NaN,5.0,NaN,5.0,3.0,4.0,4.0,NaN,3.0,NaN


In [90]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

In [91]:
corr_df

userId    userId 
28866.0   67756.0   -0.936065
60562.0   37121.0   -0.915003
34103.0   80593.0   -0.898718
62575.0   21398.0   -0.896612
117826.0  48416.0   -0.883969
                       ...   
96952.0   34350.0    0.971667
32344.0   68063.0    0.977387
28941.0   28941.0    1.000000
16562.0   41005.0    1.000000
725.0     43765.0         NaN
Length: 3821114, dtype: float64

In [92]:
corr_df = pd.DataFrame(corr_df, columns = ["corr"])
corr_df

,,corr
userId,userId,
28866.0,67756.0,-0.936065
60562.0,37121.0,-0.915003
34103.0,80593.0,-0.898718
62575.0,21398.0,-0.896612
117826.0,48416.0,-0.883969
...,...,...
96952.0,34350.0,0.971667
32344.0,68063.0,0.977387
28941.0,28941.0,1.000000


In [93]:
corr_df.index.names = ["userId_1", "userId_2"]

In [94]:
corr_df = corr_df.reset_index()

In [95]:
corr_df.head()

,userId_1,userId_2,corr
0,28866.0,67756.0,-0.936065
1,60562.0,37121.0,-0.915003
2,34103.0,80593.0,-0.898718
3,62575.0,21398.0,-0.896612
4,117826.0,48416.0,-0.883969


In [96]:
top_users = corr_df[(corr_df["userId_1"] == random_user) & 
                    (corr_df["corr"] >= 0.65)][["userId_2", "corr"]]. \
                    reset_index(drop=True)

In [97]:
top_users

,userId_2,corr
0,132307.0,0.660726
1,42696.0,0.661268
2,1362.0,0.665461
3,124546.0,0.669585
4,73844.0,0.670105
5,66603.0,0.673862
6,58705.0,0.680166
7,131285.0,0.681491
8,125901.0,0.681795
9,91954.0,0.696581


In [98]:
top_users.sort_values("corr", ascending=False)

,userId_2,corr
27,28941.0,1.000000
26,45158.0,0.800749
25,101628.0,0.790405
24,127259.0,0.763925
23,9783.0,0.747942
22,93089.0,0.738536
21,136259.0,0.737177
20,136259.0,0.737177
19,67346.0,0.732943
18,67346.0,0.732943


In [101]:
top_users.rename(columns={"userId_2": "userId"}, inplace=True)

In [104]:
rating = pd.read_csv("/kaggle/input/movielense20m/rating.csv")
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [106]:
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how="inner")
top_users_ratings

,userId,corr,movieId,rating
0,132307.0,0.660726,11,4.0
1,132307.0,0.660726,17,5.0
2,132307.0,0.660726,19,4.0
3,132307.0,0.660726,25,5.0
4,132307.0,0.660726,36,5.0
...,...,...,...,...
13680,28941.0,1.000000,534,5.0
13681,28941.0,1.000000,586,3.0
13682,28941.0,1.000000,588,2.0
13683,28941.0,1.000000,593,1.0


In [107]:
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]

### Skor Hesaplama

In [117]:
top_users_ratings["weighted_rating"] = top_users_ratings["corr"] * top_users_ratings["rating"]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
recommendation_df = top_users_ratings.groupby("movieId").agg({"weighted_rating": "mean"})

In [119]:
recommendation_df

,weighted_rating
movieId,
1,2.205824
2,1.463950
3,1.181772
4,1.642769
5,1.151408
...,...
56367,2.565300
56508,2.931772
56788,2.565300


In [120]:
recommendation_df.reset_index(inplace=True)

In [122]:
recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values

,movieId,weighted_rating
26,30,3.952023
45,53,3.952023
183,242,3.952023
201,263,3.649974
257,326,3.952023
394,501,3.679739
799,1201,3.642581
848,1257,3.603372
920,1348,3.692678
1543,2496,3.642581


In [123]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5]. \
sort_values("weighted_rating", ascending=False)

In [124]:
movies_to_be_recommend.head()

,movieId,weighted_rating
26,30,3.952023
257,326,3.952023
183,242,3.952023
45,53,3.952023
920,1348,3.692678


In [125]:
movie = pd.read_csv("/kaggle/input/movielense20m/movie.csv")
movies_to_be_recommend.merge(movie[["movieId", "title"]])

,movieId,weighted_rating,title
0,30,3.952023,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
1,326,3.952023,To Live (Huozhe) (1994)
2,242,3.952023,Farinelli: il castrato (1994)
3,53,3.952023,Lamerica (1994)
4,1348,3.692678,"Nosferatu (Nosferatu, eine Symphonie des Graue..."
5,501,3.679739,Naked (1993)
6,25850,3.664714,Holiday (1938)
7,8128,3.664714,Au revoir les enfants (1987)
8,26394,3.664714,"Turning Point, The (1977)"
9,7585,3.664714,Summertime (1955)
